# Results (table)

In [1]:
import os
import numpy as np
import pandas as pd
from dataloader import load_raw, create_datasets
from measures import compute_intermittent_indicators, quantile_loss_sample, quantile_loss

In [2]:
results_path = os.path.join(os.path.expanduser("~/switchdrive"), "iTS", "trained_models")
experiments_name = [folder for folder in os.listdir(results_path) 
                    if os.path.isdir(os.path.join(results_path, folder)) and os.path.exists(os.path.join(results_path, folder, 'metrics.json'))]
baseline_path = os.path.join(os.path.expanduser("~/switchdrive"), "iTS", "trained_models_baselines")
baselines_name = [folder for folder in os.listdir(baseline_path) 
                  if os.path.isdir(os.path.join(baseline_path, folder)) and os.path.exists(os.path.join(baseline_path, folder, 'metrics.json'))]

def compute_table(subset, aggf=np.mean, scale=None):
    tables = []
    for dataset, model, N in zip(["M5","OnlineRetail","carparts","RAF","Auto"],["transformer","deepAR","deepAR","deepAR","deepAR","deepAR"],[5,10,10,10,10]):

        scaling = "mean-demand"
        distr = ['negbin','tweedie']

        data_raw, data_info = load_raw(dataset_name=dataset, datasets_folder_path=os.path.join("..","data"))
        datasets = create_datasets(data_raw, data_info)
        adi, cv2 = compute_intermittent_indicators(data_raw, data_info['h'])
        if subset == "intermittent":
            filter, filter_label = np.logical_and(adi >= 1.32, cv2 < .49), "intermittent"
        elif subset == "intermittent_and_lumpy":
            filter, filter_label = adi >= 1.32, "intermittent_and_lumpy"
        elif subset == "all":
            filter, filter_label = np.tile(True, adi.size), "all"

        experiments_name_sub = [x for x in experiments_name if x.split('__')[0] == model and x.split('__')[1] == dataset and x.split('__')[3] == scaling and x.split('__')[2] in distr]
        sub = pd.DataFrame([x.split('__') for x in experiments_name_sub], columns=['model','dataset','distr','scaling','datetime'])

        baselines_name_sub = [x for x in baselines_name if x.split('__')[1] == dataset]

        quantiles = [0.5,0.8,0.9,0.95, 0.99]
        tmp = np.empty(shape=(len(datasets['test']), len(datasets['valid'][0]['target']), len(quantiles)))
        for i in range(len(datasets['test'])):
            for j in range(len(quantiles)):
                tmp[i, :] = np.round(np.quantile(datasets['valid'][i]['target'], q=quantiles))
        res_base_scale_tmp = []
        for i in range(len(datasets['test'])):
            res_base_scale_tmp.append(quantile_loss(np.array(datasets['valid'][i]['target']).reshape(1,-1), tmp[i].reshape(1,tmp[i].shape[0],tmp[i].shape[1]), quantiles, avg=False))
        res_base_scale = {}
        for q in ['QL50','QL80','QL90','QL95','QL99']:
            res_base_scale[q] = np.mean(np.vstack([res_base_scale_tmp[i][q] for i in range(len(datasets['test']))]), axis=1)[filter]

        fscale = lambda x, q: x / res_base_scale[q][:, np.newaxis] if scale else x

        res_base = {}
        for exp in baselines_name_sub:
            a = np.load(os.path.join(baseline_path, exp,"actuals.npy"))[filter]
            qfc = np.load(os.path.join(baseline_path, exp,"qforecasts.npy"))[filter]
            ql = quantile_loss(a, qfc, quantiles, avg=False)
            res_base[exp.split('__')[0]] = {}
            for q in ['QL50','QL80','QL90','QL95','QL99']:
                res_base[exp.split('__')[0]][q] = aggf(fscale(ql[q], q))

        res = {}
        ql_full = {}
        for d in distr:
            sub_ = sub[sub.distr == d]
            sub_paths = ["__".join(x.tolist()) for x in sub_.values]
            assert len(sub_paths) == N

            ql_all = {}
            ql_full[d] = {}
            for exp in sub_paths:
                forecasts = np.load(os.path.join(results_path,exp,'forecasts.npy'))
                actuals = np.load(os.path.join(results_path,exp,'actuals.npy'))
                ql = quantile_loss_sample(actuals[filter,:], forecasts[filter,:,:], avg=False)
                for q in ['QL50','QL80','QL90','QL95','QL99']:
                    if not q in ql_all: ql_all[q] = []
                    ql_all[q].append(aggf(fscale(ql[q], q)))
                    if not q in ql_full[d]: ql_full[d][q] = []
                    ql_full[d][q].append(ql[q])
            ql_all_means = {key: np.mean(values) for key, values in ql_all.items()}
            ql_all_std = {key: np.std(values) for key, values in ql_all.items()}

            res[d] = {'mean':ql_all_means, 'std':ql_all_std}

        tweedie_wins_mean = {}
        tweedie_wins_std = {}
        negbin_wins_mean = {}
        negbin_wins_std = {}
        for q in ['QL50','QL80','QL90','QL95','QL99']:
            tweedie_wins_mean[q] = []
            a = np.stack(ql_full['negbin'][q])
            b = np.stack(ql_full['tweedie'][q])
            a = np.mean(a, axis=2)
            b = np.mean(b, axis=2)
            tmp = [np.mean(b_ < a_)*100 for a_, b_ in zip(a, b)]
            tweedie_wins_mean[q] = np.mean(tmp)
            tweedie_wins_std[q] = np.std(tmp)
            tmp = [np.mean(b_ > a_)*100 for a_, b_ in zip(a, b)]
            negbin_wins_mean[q] = np.mean(tmp)
            negbin_wins_std[q] = np.std(tmp)

        def out(m, std=None, precision=3):
            if std:
                return [str(np.round(m[k],3))+'±'+str(np.round(std[k],3)) for k in m.keys()]
            return [str(np.round(m[k],3)) for k in m.keys()]

        print(dataset)
        df = pd.DataFrame(data=[out(res_base['EmpQ']),
                        out(res_base['iETS']),
                        out(res['negbin']['mean'], res['negbin']['std']),
                        out(res['tweedie']['mean'], res['tweedie']['std']),
                        out(negbin_wins_mean, negbin_wins_std), out(tweedie_wins_mean, tweedie_wins_std)], 
                        index=['empQ','iETS',model+"-negbin",model+"-tweedie","negbin wins","tweedie wins"], 
                        columns=["QL50","QL80","QL90","QL95","QL99"])
        tables.append(df)
        def highlight_min(s):
            tmp = np.array([float(x.split('±')[0]) for x in s.values[:-2]])
            is_min = tmp == np.min(tmp)
            aaa = ['background-color: green' if v else '' for v in is_min]
            tmp = np.array([float(x.split('±')[0]) for x in s.values[-2:]])
            is_max = tmp == np.max(tmp)
            return  aaa + ['background-color: gold; color:black' if v else 'background-color: khaki; color:black' for v in is_max]
        display(df.style.apply(highlight_min, axis=0))
    return tables

In [ ]:
_ = compute_table(subset="intermittent_and_lumpy", aggf=np.mean)

In [ ]:
_ = compute_table(subset="intermittent", aggf=np.mean, scale="EmpQ")

In [ ]:
_ = compute_table(subset="intermittent_and_lumpy", aggf=np.mean, scale="EmpQ")